In [20]:
from utilities import *
from core.initial import Insertions
from docplex.cp.model import *

In [21]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')
model = CSPModel(d)
model.build_model()

model.data.head()

,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
0,Fix Station,Fix Station,NaN,05:00,40,300,340
1,Fix Station,Fix Station,NaN,05:20,40,320,360
2,Fix Station,Fix Station,NaN,05:55,40,355,395
3,Fix Station,Fix Station,NaN,06:20,44,380,424
4,Fix Station,Fix Station,NaN,06:40,44,400,444


In [ ]:
# initial = Insertions(model)
# initial.solve()

In [ ]:
# trips = [interval_var(start=trip.start_time,
#                       end=trip.end_time,
#                       size=trip.duration,
#                       name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


In [22]:
ntrips = len(model.trips)
nduties = len(model.trips)

sub = CpoModel(name="Pricing_Subproblem")


min_start = model.data[start_time].min()
max_start = model.data[start_time].max()
min_end = model.data[end_time].min()
max_end = model.data[end_time].max()

duties = [interval_var(start=(min_start, max_start),
                       end=(min_end, max_end),
                       size=model.constraints.shift_span,
                       name=f"Duty_{i}",
                       optional=True)
          for i in range(nduties)]

trip2trip = integer_var_list(size=ntrips,
                             min=0,
                             max=ntrips + 1,
                             name='Trip2Trip')

trip2duty = integer_var_list(size=ntrips,
                             min=0,
                             max=nduties,
                             name='Trip2Duty')

start_times = [[integer_var(min=0,
                           max=model.constraints.shift_span,
                           name=f"StartTime-{i}-{j}")
               for j in range(nduties)] for i in range(ntrips)]

cdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.continuous_driving,
                       name="CDT")

tdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.total_driving,
                       name="TDT")

In [23]:
for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_time_arr[i] <= model.start_time_arr[j]))

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_loc_arr[i] <= model.start_loc_arr[j]))


for i in range(ntrips):
    sub.add(trip2trip[i] != i)

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(trip2trip[i] == j, trip2duty[i] == trip2duty[j]))

In [24]:
def report_solution(cpsol: CpoSolveResult):
    trips_per_duty = {}
    for i in range(ntrips):
        _out = f"{i:>2} -> {cpsol[trip2trip[i]]} | Duty: {cpsol[trip2duty[i]]}"
        print(_out)

        duty_id = cpsol[trip2duty[i]]

        if duty_id in trips_per_duty:
            trips_per_duty[duty_id].append(i)
        else:
            trips_per_duty[duty_id] = []
            trips_per_duty[duty_id].append(i)

    print(f'\n\nTotal Duties: {len(trips_per_duty.keys())}')

    for duty_id, duty_trips in trips_per_duty.items():
        df_trips = model.data.loc[duty_trips]

        span = df_trips[end_time].max() - df_trips[start_time].min()

        print(
            f'\n\n>>> Duty {duty_id} - Trips: {len(duty_trips)} - Drive Time: {df_trips[trip_duration].sum()} - Shift Span: {span}\n')
        print(df_trips.to_markdown())

In [25]:
msol = sub.solve()

report_solution(msol)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Satisfiability problem - 116 variables, 10150 constraints
 ! Initial process time : 0.06s (0.06s extraction + 0.01s propagation)
 !  . Log search space  : 600.2 (before), 600.2 (after)
 !  . Memory usage      : 2.5 MB (before), 2.5 MB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                     69  0.09s        1         4  = Trip2Duty_13
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 272
 ! Number of fails        : 0
 ! Total memory usage     : 16.0 MB (16.0 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.09s (0.03s engine + 0.06s extraction)
 ! Search speed 

In [ ]:
type(msol[trip2duty[0]])

In [ ]:
trips = [8, 13, 30, 43, 46, 47, 49, 50, 52]

In [ ]:
model.data.loc[trips]

In [ ]:
for i in range(ntrips):
    sub.add(trip2trip[i] != 0)
    # sub.add(sum([trip2trip[j] == i for j in range(ntrips+1)]) == 1)

In [ ]:
for i in range(ntrips-1):
    for j in range(i + 1, ntrips):
        sub.add(end_before_start(trips[i], trips[j]))

In [ ]:
has_break = [[binary_var(name=f'Break_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]
has_rest = [[binary_var(name=f'Rest_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]

In [ ]:
trip_seq = sequence_var(trips)

In [ ]:
td_matrix = {}
for trip in model.trips:
    for duty in range(ntrips):
        td_matrix[(trip.ID, duty)] = binary_var(name=f"T{trip.ID}-D{duty}")

In [ ]:
end_of(trips[0]).equals(trips[1])

In [ ]:
INTERVAL_MAX